In [1]:
# СТРОИМ НЕЙРОННУЮ СЕТЬ СО СКРЫТЫМ ReLU-СЛОЕМ ДЛЯ РАСПОЗНАВАНИЯ 
# РУКОПИСНЫХ ЦИФР

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

#Sets the threshold for what messages will be logged.
old_v = tf.logging.get_verbosity()

# able to set the logging verbosity to either DEBUG, INFO, WARN,ERROR  
# or FATAL. Here its ERROR
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
x = tf.placeholder(tf.float32, [None, 784])

# ... чтобы наша модель имела... название нейронной сети, нужно добавить
# в нее скрытый слой. В качестве функции активации возьмем все тот же
# ReLU. Нужно инициировать веса и свободный член для этого слоя;
# будем в этом примере использовать 100 нейронов на скрытом слое:
W_relu = tf.Variable(tf.truncated_normal([784,100], stddev=0.1))
b_relu = tf.Variable(tf.truncated_normal([100], stddev=0.1))
# На этот раз мы инициализируем переменные не нулями, а небольшими
# случайными значениями. Функция tf.truncated_normal возвращает
# значения, порожденные нормально распределенной случайной величиной
# с фиксированными мат.ожиданием и дисперсией; однако при этом
# значения, вышедшие за пределы итервала +-2сигма от среднего,
# выбираются заново, то есть распределение обрезается так, чтобы
# полностью запретить большие выбросы. Инициализация нулями в данном
# случае была бы совсем бессмысленной, потому что ReLU(0)=0, а значит,
# при инициализации нулями градиенты совсем не распространялись бы по сети.
# В нашем же случае примерно половина весов окажется отрицательной, и   
# соответствующие нейроны не будут активированы вовсе.

# ... общий вид скрытого слоя получается таким:
h = tf.nn.relu(tf.matmul(x,W_relu) + b_relu)
# Применение tf.nn.relu тоже будет покомпонентным: фактически мы просто
# применили функцию ReLU к вектору tf.matmul(x, W_relu) + b_relu

# В этой нейронной сети будет очень полезен слой дропаута...: дропаут -
# это слой, который выбрасывает (обнуляет) выходы некоторых нейронов,
# выбираемых случайно и заново для каждого обучающего примера.
# Все, что нам сейчас нужно задать - это вероятность их выбрасывания;
# для этого мы сначала создадим заглушку:
keep_probability = tf.placeholder(tf.float32)

# А дальше TensorFlow... все делает за нас...
h_drop = tf.nn.dropout(h, keep_probability)
# Теперь нейроны скрытого слоя будут участвовать в вычислениях с вероятностью
# keep_probability, а с вероятностью 1-keep_probability их выход будет 
# обнулен, и они не будут ни участвовать в предсказании для этого примера,
# ни обучаться на нем. 

# Поскольку размер внутреннего слоя отличается от входного, нам придется
# немного поменять параметры внешнего слоя и, кроме того, переписать
# заключительный softmax-слой:
#
W = tf.Variable(tf.zeros([100, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(h_drop,W) + b)

# [без изменений с предыдущей моделью - обучения логистической регрессии]
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(
    -tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.initialize_all_variables()
# При этом наша функция потель cross_entropy и оптимизатор train_step 
# не требуют никаких изменений. А вот вызвать sess.run нужноо с новым
# переметром keep_probability. Поэтому цикл обучения немного изменился:

In [3]:
# Вариант № 1: 2000 итераций с САМОПИСНОЙ функцией 
# расчета перекрестной энтропии

# Если сделать 2000 шагов в обучении нашей новой сети, мы получим
sess = tf.Session()
sess.run(init)
for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys,
                                   keep_probability: 0.5})
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: %.4f" %
     sess.run(accuracy, feed_dict={x: mnist.test.images, 
                y_:mnist.test.labels, keep_probability: 1.}))
# Отличный результат! Мы дописали еще несколько строк кода и с самыми
# минимальными изменениями смогли добавить целый новый скрытый слой
# в модель, а также уменьшить ошибку на тестовой выборке практически
# в два раза.

Accuracy: 0.9639


In [4]:
# Вариант № 2: 4000 итераций с САМОПИСНОЙ функцией 
# расчета перекрестной энтропии

# Но и это еще не все. ... нельзя ли обучить нашу сеть еще лучше,
# просто потратив на это больше времени? Но не тут-то было! Еще
# через 2000 шагов обучения точность неожиданно резко падает
# примерно до 10%
sess = tf.Session()
sess.run(init)
for i in range(4000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys,
                                   keep_probability: 0.5})
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: %.4f" %
     sess.run(accuracy, feed_dict={x: mnist.test.images, 
            y_:mnist.test.labels, keep_probability: 1.}))    

Accuracy: 0.0980


In [5]:
# Давайте разбираться, в чем причина. Вообще говоря, 10% подозрительно
# похоже на точность генератора случайных чисел при угадывании
# равномерно распределенных 10 классов; поэтому первое, что нужно 
# проверить - не сломалась ли наша модель полностью. Для этого можно,
# например, посмотреть на ее веса:
sess.run(b)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [6]:
# И действительно, элементы вектора свободных членов b перестали быть
# числами. Чаще всего это говорит о переполнении. Оно происходит, когда
# очень большие числа выходят за границы соответствующих диапазонов и
# округляются до "бесконечности" или "минус бесконечности"; чаще всего
# так получается, когда очень маленькие числа округляются до нуля,
# а потом на этот ноль что-нибудь пытаются разделить. Но откуда
# переполнение могло появиться у нас?
# Одно из "узких" мест нашего кода - функция softmax..

In [7]:
# Вариант № 3: 10 000 итераций со ВСТРОЕННОЙ функцией 
# расчета перекрестной энтропии

# К счастью, создатели TensorFlow знали об этих трудностях и специально
# подготовили функцию, которая обходим все подводные камни. Давайте
# использовать вместо самописных функций. Зададим промежуточный тензор
logit = tf.matmul(h_drop, W) + b

# Теперь мы не будем сами применять softmax или считать перекрестную
# энтропию, а воспользуемся готовой функцией:
# [аргументы должны быть именованы, иначе не работает]
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                               (logits=logit, labels=y_))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.initialize_all_variables()

# Больше ничего менять не надо; давайте снова запустим код, теперь уже 
# сделав 10 000 шагов обучения, и посмотрим на результат:
sess = tf.Session()
sess.run(init)
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys,
                                   keep_probability: 0.5})
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: %.4f" %
     sess.run(accuracy, feed_dict={x: mnist.test.images, 
                y_:mnist.test.labels, keep_probability: 1.}))
# Неплохо! Уже 97,4% - теперь наша модель ошибается (на тестовой выборке,
# естественно) всего один раз из сорока. 

Accuracy: 0.9737
